# Системы хранения и обработки данных. Домашнее задание 2.
## Присяжнюк Артем Сергеевич

# Подготовка

In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2 as pg

In [274]:
conn = pg.connect(dbname="postgres", host="localhost", user="postgres", password="password", port=5432)
cursor = conn.cursor()

# Задание 1

## Создание таблиц

In [198]:
cursor.execute("""
               create table transaction (
                   transaction_id int4 NOT NULL PRIMARY KEY
                   ,product_id int4
                   ,customer_id int4 
                   ,transaction_date varchar
                   ,online_order varchar
                   ,order_status varchar
                   ,list_price float4
                   ,brand varchar
                   ,product_line varchar
                   ,product_class varchar
                   ,product_size varchar
                   ,standard_cost float4                 
               )
               """)

In [199]:
cursor.execute("""
               create table customer (
                   customer_id int4 NOT NULL PRIMARY KEY
                   ,first_name varchar
                   ,last_name varchar
                   ,gender varchar
                   ,dob varchar
                   ,job_title varchar
                   ,job_industry_category varchar
                   ,wealth_segment varchar
                   ,deceased_indicator varchar
                   ,owns_car varchar
                   ,property_valuation int4
                   ,address varchar
                   ,postcode varchar
                   ,state varchar
                   ,country varchar                  
               )
               """)

In [200]:
conn.commit()

## Обработка данных

In [201]:
import pandas as pd

df_transaction = pd.read_csv("data/transaction.csv", sep=';')
df_transaction.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"
2,3,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,"1793,43","248,82"
3,4,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,"1198,46","381,10"
4,5,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,"1765,3","709,48"


In [202]:
df_transaction.standard_cost = df_transaction.standard_cost.apply(lambda x: float(str(x).replace(',', '.')))
df_transaction.list_price = df_transaction.list_price.apply(lambda x: float(str(x).replace(',', '.')))

In [203]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    20000 non-null  int64  
 1   product_id        20000 non-null  int64  
 2   customer_id       20000 non-null  int64  
 3   transaction_date  20000 non-null  object 
 4   online_order      19640 non-null  object 
 5   order_status      20000 non-null  object 
 6   brand             19803 non-null  object 
 7   product_line      19803 non-null  object 
 8   product_class     19803 non-null  object 
 9   product_size      19803 non-null  object 
 10  list_price        20000 non-null  float64
 11  standard_cost     19803 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 1.8+ MB


In [204]:
df_customer = pd.read_csv("data/customer.csv", sep=';')
df_customer.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


In [205]:
df_customer['deceased_indicator'].unique(), df_customer['owns_car'].unique()

(array(['N', 'Y'], dtype=object), array(['Yes', 'No'], dtype=object))

In [206]:
df_customer['gender'].unique()

array(['F', 'Male', 'Female', 'U', 'Femal', 'M'], dtype=object)

In [207]:
df_customer['gender'].replace({'Female': 'F', 'Femal': 'F', 'Male': 'M'}, inplace=True)

In [208]:
df_customer['country'].unique(), df_customer['state'].unique()

(array(['Australia'], dtype=object),
 array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object))

In [209]:
df_customer['state'].replace({'New South Wales': 'NSW', 'Victoria': 'VIC'}, inplace=True)

In [210]:
df_customer.rename({'DOB': 'dob'}, axis=1, inplace=True)

In [211]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            4000 non-null   int64 
 1   first_name             4000 non-null   object
 2   last_name              3875 non-null   object
 3   gender                 4000 non-null   object
 4   dob                    3913 non-null   object
 5   job_title              3494 non-null   object
 6   job_industry_category  3344 non-null   object
 7   wealth_segment         4000 non-null   object
 8   deceased_indicator     4000 non-null   object
 9   owns_car               4000 non-null   object
 10  address                4000 non-null   object
 11  postcode               4000 non-null   int64 
 12  state                  4000 non-null   object
 13  country                4000 non-null   object
 14  property_valuation     4000 non-null   int64 
dtypes: int64(3), object(1

In [212]:
transaction_names = [
    'transaction_id', 'transaction_date',
    'online_order', 'order_status',
    'list_price', 
    'product_id', 'customer_id',
    'brand',
    'product_line', 'product_class',
    'product_size', 'standard_cost'
    
]

customer_names = [
    'customer_id', 'first_name', 'last_name',
    'gender', 'dob',
    'job_title', 'job_industry_category',
    'wealth_segment', 'property_valuation', 'owns_car',
    'deceased_indicator',
    'address', 'country',
    'state', 'postcode'
]

In [213]:
df_transaction.shape, df_transaction.transaction_id.nunique()

((20000, 12), 20000)

In [214]:
df_customer.shape, df_customer.customer_id.nunique()

((4000, 15), 4000)

In [162]:
# nat_mask = df_customer.dob.apply(lambda x: x is pd.NaT)
# na_mask = df_customer.dob.isna()
# df_customer.loc[nat_mask | na_mask, "dob"] = pd.to_datetime(0) # because postgresql does not accept NaT

In [151]:
%pip install tqdm

  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [215]:
from tqdm import tqdm

for _, row_t in tqdm(df_transaction.iterrows()):
    cursor.execute(f"""
                insert into transaction ({','.join(transaction_names)})
                values (%s{', %s' * (len(transaction_names)-1)})
                on conflict do nothing
                """, row_t[transaction_names])

20000it [00:20, 960.03it/s] 


In [227]:
for _, row_c in tqdm(df_customer.iterrows()):
    cursor.execute(f"""
                insert into customer ({','.join(customer_names)})
                values (%s{', %s' * (len(customer_names)-1)})
                on conflict do nothing
                """, row_c[customer_names])

4000it [00:04, 813.92it/s]


In [228]:
conn.commit()

# Задание 2

### 2.1

In [278]:
conn.rollback()

In [279]:
cursor.execute(f"""
                select distinct brand
                from transaction
                where standard_cost > 1500
                """)
print(cursor.fetchall())

[('NaN',), ('OHM Cycles',), ('Trek Bicycles',), ('Solex',), ('Giant Bicycles',)]


In [104]:
df_transaction.brand.unique()

array(['Solex', 'Trek Bicycles', 'OHM Cycles', 'Norco Bicycles',
       'Giant Bicycles', 'WeareA2B', nan], dtype=object)

In [103]:
df_transaction.loc[df_transaction['standard_cost'] > 1500, 'brand'].unique()

array(['Trek Bicycles', 'Giant Bicycles', 'OHM Cycles', 'Solex'],
      dtype=object)

### 2.2

In [265]:
cursor.execute(f"""
                select *
                from transaction
                where order_status = 'Approved' 
                and to_timestamp(transaction_date, 'DD.MM.YYYY') between '2017-04-01'::timestamp and '2017-04-09'::timestamp
                order by transaction_date
                """)
result = cursor.fetchall()
print(len(result))
print(result)

531
[(15576, 68, 2820, '01.04.2017', 'true', 'Approved', 1636.9, 'OHM Cycles', 'Standard', 'medium', 'medium', 44.71), (16621, 90, 2072, '01.04.2017', 'false', 'Approved', 363.01, 'Norco Bicycles', 'Standard', 'low', 'medium', 290.41), (2146, 8, 3096, '01.04.2017', 'true', 'Approved', 1703.52, 'Solex', 'Road', 'medium', 'small', 1516.13), (15488, 64, 906, '01.04.2017', 'true', 'Approved', 1977.36, 'Giant Bicycles', 'Standard', 'high', 'small', 1759.85), (5922, 8, 1831, '01.04.2017', 'true', 'Approved', 1703.52, 'Solex', 'Road', 'medium', 'small', 1516.13), (14684, 18, 2424, '01.04.2017', 'false', 'Approved', 575.27, 'Solex', 'Standard', 'medium', 'medium', 431.45), (9378, 19, 1053, '01.04.2017', 'true', 'Approved', 12.01, 'OHM Cycles', 'Road', 'high', 'large', 7.21), (2370, 45, 2526, '01.04.2017', 'false', 'Approved', 441.49, 'Solex', 'Standard', 'medium', 'medium', 84.99), (6717, 30, 1985, '01.04.2017', 'true', 'Approved', 748.17, 'Solex', 'Standard', 'high', 'medium', 448.9), (14394,

### 2.3

In [120]:
df_customer.job_industry_category.unique()

array(['Health', 'Financial Services', 'Property', 'IT', nan, 'Retail',
       'Argiculture', 'Manufacturing', 'Telecommunications',
       'Entertainment'], dtype=object)

In [229]:
cursor.execute(f"""
                select job_title
                from customer
                where job_industry_category in ('IT', 'Financial Services')
                and starts_with(job_title, 'Senior')
                """)
result = cursor.fetchall()
print(len(result))
print(result)

81
[('Senior Quality Engineer',), ('Senior Cost Accountant',), ('Senior Financial Analyst',), ('Senior Financial Analyst',), ('Senior Sales Associate',), ('Senior Financial Analyst',), ('Senior Financial Analyst',), ('Senior Financial Analyst',), ('Senior Cost Accountant',), ('Senior Financial Analyst',), ('Senior Cost Accountant',), ('Senior Cost Accountant',), ('Senior Financial Analyst',), ('Senior Cost Accountant',), ('Senior Developer',), ('Senior Financial Analyst',), ('Senior Cost Accountant',), ('Senior Financial Analyst',), ('Senior Financial Analyst',), ('Senior Financial Analyst',), ('Senior Cost Accountant',), ('Senior Cost Accountant',), ('Senior Cost Accountant',), ('Senior Developer',), ('Senior Financial Analyst',), ('Senior Sales Associate',), ('Senior Cost Accountant',), ('Senior Financial Analyst',), ('Senior Financial Analyst',), ('Senior Financial Analyst',), ('Senior Financial Analyst',), ('Senior Quality Engineer',), ('Senior Financial Analyst',), ('Senior Cost A

### 2.4

In [230]:
cursor.execute(f"""
                select distinct brand
                from transaction as t left join customer as c on t.customer_id = c.customer_id 
                where job_industry_category = 'Financial Services'
                """)
result = cursor.fetchall()
print(len(result))
print(result)

7
[('Giant Bicycles',), ('NaN',), ('Norco Bicycles',), ('OHM Cycles',), ('Solex',), ('Trek Bicycles',), ('WeareA2B',)]


### 2.5

In [243]:
df_transaction.online_order.unique()

array([False, True, nan], dtype=object)

In [258]:
cursor.execute(f"""
                select distinct c.first_name, c.last_name
                from transaction as t left join customer as c on t.customer_id = c.customer_id
                where brand in ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
                and coalesce(nullif(online_order, 'NaN'), 'False')::boolean is True
                limit 10
                """)
result = cursor.fetchall()
print(len(result))
print(result)

10
[('Brett', 'McCalister'), ('Joannes', 'Harbron'), ('Byrom', 'Draisey'), ('Terrijo', 'Merlin'), ('Melamie', 'Hardman'), ('Kerry', 'Pashenkov'), ('Banky', 'Baudichon'), ('Drake', 'Riguard'), ('Venus', 'Sketch'), ('Maressa', 'Kondrachenko')]


### 2.6

In [259]:
cursor.execute(f"""
                select distinct c.first_name, c.last_name
                from customer as c left join transaction as t on c.customer_id = t.customer_id
                where t.customer_id is NULL
                """)
result = cursor.fetchall()
print(len(result))
print(result)

507
[('Costa', 'Sleightholm'), ('Barnie', 'Blenkensop'), ('Enriqueta', 'Waterhowse'), ('Ali', 'Naris'), ('Davie', 'Moscon'), ('Chicky', 'Butterfield'), ('Luca', 'Cabotto'), ('Donavon', 'Ainsby'), ('Ado', 'Gailor'), ('Wadsworth', 'Ruselin'), ('Zarla', 'Seyers'), ('Rica', 'Corrado'), ('Travis', 'Kaszper'), ('Mahmoud', 'Ligerton'), ('Jessie', 'Wards'), ('Hadria', 'Moles'), ('Josepha', 'Finder'), ('Marinna', 'Ors'), ('Troy', 'Blaydes'), ('Shay', 'Rayer'), ('Peder', 'Frotton'), ('Blancha', 'Baldi'), ('Lemuel', 'Gerram'), ('Myrtia', 'NaN'), ('Ephrem', 'Hollerin'), ('Elyssa', 'De Witt'), ('Nicolas', 'Burdass'), ('Olivie', 'Nazair'), ('Erik', 'Kenneford'), ('Caryn', 'Padbury'), ('Kacie', 'Stanbrooke'), ('Dolf', 'Serridge'), ('Kennie', 'Baszniak'), ('Berti', 'Arnould'), ('Cassandra', 'Cescotti'), ('Tuckie', 'NaN'), ('Christi', 'Gilluley'), ('Cornie', 'Hillhouse'), ('Willow', 'Cutmere'), ('Di', 'Borsnall'), ('Orel', 'Joncic'), ('Jemima', 'Izaac'), ('Griffith', 'Minor'), ('Perry', 'Lampert'), ('H

### 2.7

In [262]:
cursor.execute(f"""
                select distinct c.first_name, c.last_name
                from transaction as t left join customer as c on t.customer_id = c.customer_id
                where c.job_industry_category = 'IT'
                and standard_cost = (
                    select max(standard_cost)
                    from transaction
                )
                """)
result = cursor.fetchall()
print(len(result))
print(result)

16
[('Birk', 'Elphey'), ('Carolann', 'Ottiwill'), ('Caterina', 'Scedall'), ('Goldarina', 'Rzehorz'), ('Jephthah', 'Bachmann'), ('Keeley', 'Kruger'), ('Moritz', 'Brognot'), ('Nickolas', 'Guittet'), ('Obed', 'Pauler'), ('Pieter', 'Gadesby'), ('Reggie', 'Broggetti'), ('Robin', 'Crack'), ('Shandie', 'Sprigg'), ('Sharla', 'Creebo'), ('Valaria', 'Hugh'), ('Wayne', 'Woodfin')]


### 2.8

In [271]:
cursor.execute(f"""
                select distinct c.first_name, c.last_name
                from transaction as t left join customer as c on t.customer_id = c.customer_id
                where (c.job_industry_category in ('IT', 'Health'))
                and (t.order_status = 'Approved')
                and (to_timestamp(transaction_date, 'DD.MM.YYYY') between '2017-07-07'::timestamp and '2017-07-17'::timestamp)
                """)
result = cursor.fetchall()
print(len(result))
print(result)

115
[('Ahmed', 'Edmondson'), ('Aldous', 'Cubin'), ('Aloisia', 'Shawel'), ('Alwin', 'Cuberley'), ('Andreas', 'Bazely'), ('Angelo', 'NaN'), ('Annabella', 'Hebron'), ('Ardelle', 'NaN'), ('Arliene', 'Bartolijn'), ('Basilius', 'Coupe'), ('Betsy', 'Cosh'), ('Birk', 'Elphey'), ('Booth', 'Birkin'), ('Brenn', 'Bacon'), ('Bret', 'Ivakhnov'), ('Bronson', 'Plowman'), ('Bryn', 'Rycraft'), ('Burr', 'Gerb'), ('Cameron', 'Letherbury'), ('Carmela', 'Jesper'), ('Caterina', 'Scedall'), ('Cathlene', 'Bellas'), ('Cinderella', 'Haslen'), ('Clint', 'Krause'), ('Conroy', 'Healy'), ('Corny', 'Millott'), ('Creight', 'Deery'), ('Damara', 'Wheatland'), ("D'arcy", 'Slay'), ('Daron', 'Umpleby'), ('Dawna', 'McCullen'), ('Ddene', 'Burleton'), ('Deeanne', 'Durtnell'), ('Dennie', 'Eunson'), ('Durante', 'Reddan'), ('Elinor', 'Biggerstaff'), ('Elka', 'Crossgrove'), ('Emelda', 'Gerler'), ('Erastus', 'Payley'), ('Ezra', 'Steffan'), ('Fair', 'Dewen'), ('Fannie', 'Devil'), ('Fee', 'Zellmer'), ('Ford', 'Groucock'), ('Frederic

# Завершение работы

In [272]:
cursor.close()
conn.close()